<a href="https://colab.research.google.com/github/hamzamusht550/Amazon-Clone-Frontend/blob/main/MKV_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#@title Install Dependencies

from IPython.display import clear_output
from google.colab import drive
from google.colab import auth

!apt-get install -y mediainfo ffmpeg mkvtoolnix aria2
!pip install patool pymediainfo ffmpeg-python pyrogram tgcrypto nest_asyncio

drive.mount('/content/drive')
auth.authenticate_user()
clear_output()

In [ ]:

#@title MKV Merger

MKV_FILES = "https://mixologystreambot-6bb897237317.herokuapp.com/36917/%5BToonworld4all%5D+Solo+Leveling+S02E01+720p+x265+10bit+WEB-DL+.mkv?hash=AgADeB, https://mixologystreambot-6bb897237317.herokuapp.com/36923/Secret+Level+S1+2024+E9-15+Dual+Audio+Hindi+-+English+Comple.mkv?hash=AgAD4A"  # @param {type: "string"}
#MKV_FILES = "https://mixologystreambot-6bb897237317.herokuapp.com/36693/114b80ac782c4e8583c7016c9f83fe13.mkv?hash=AgADlA, https://mixologystreambot-6bb897237317.herokuapp.com/36695/60656c2c21894946bade447fea690315.mkv?hash=AgADlQ"  # @param {type: "string"}
OUTPUT_DIR = ""  #@param {type: "string"}
MAX_FileSize = 2  #@param {type: "number"}
JUST_DOWNLOAD= False  #@param {type: "boolean"}
# Delete_Extracted_Data = False  #@param {type: "boolean"}
##@markdown ***Start 00:00:00 / End 00:00:10***
# trim_start = ""  #@param {type: "string"}
# trim_end = ""  #@param {type: "string"}

from IPython.display import clear_output
import subprocess
import os
import re
import requests
import patoolib
from google.colab import drive
from zipfile import ZipFile
import ffmpeg
from pymediainfo import MediaInfo
import mimetypes
# Mount Google Drive
drive.mount('/content/drive')
clear_output()


def deleteFolder(input_folder):
    !rm -rf "{input_folder}"
    print(f"Folder Deleted: {input_folder}")

def strToList(listStr, separator=","):
    listStr = ' '.join(listStr.split()) # Remove extra spaces from String
    myList = [_.strip() for _ in listStr.split(separator)]
    myList = [_ for _ in myList if _ and _.strip()] # Remove empty items from list
    showAllURLs = "\n".join(f"{i+1}. {ele}" for i, ele in enumerate(myList))
    print(showAllURLs)
    return myList

def extract_file_info(file_path):
    # parent_dir_path = os.path.dirname(file_path)
    # file_name = os.path.basename(file_path)
    parent_dir_path, file_name = os.path.split(file_path)
    _name, _ext = os.path.splitext(file_name)
    return parent_dir_path,  file_name, _name, _ext[1:]

def getMKVFilesList(folder_path, relative=0):
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mkv'):
                full_path = os.path.join(root, file)
                if relative == 1:
                    full_path = os.path.relpath(full_path, folder_path)
                all_files.append(full_path)

    all_files.sort()
    return all_files


def extractArchive(Archive_File, extracted_folder=''):
    if not extracted_folder:
        extracted_folder = os.path.splitext(Archive_File)[0]
    if os.path.exists(extracted_folder):
        deleteFolder(extracted_folder)
    os.makedirs(extracted_folder, exist_ok=True)
    print(f"\nExtracting archive to: {extracted_folder}")
    patoolib.extract_archive(Archive_File, outdir=extracted_folder)
    print(f"Extraction complete: {extracted_folder}\n")
    return extracted_folder


def runCommand(cmd, showLog=0):
    if showLog == 1:
        # Print logs in real-time, if 2nd param is 1
        print(f"Running command: {cmd}")
        process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in iter(process.stdout.readline, b''):
            print(line.decode("utf-8"), end="")

        process.stdout.close()
        process.wait()
    else:
        process = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return process.returncode


def getResponseFileName(url):
    # Define the maximum filename length (commonly 255 characters)
    MAX_FILENAME_LENGTH = 255

    # Get the response from the URL
    response = requests.get(url, stream=True)

    # Extract the filename from the Content-Disposition header
    filename = response.headers.get('content-disposition', '')
    if 'filename=' in filename:
        filename = filename.split("filename=")[-1].strip('"')
    else:
        return ""

    # Define a regex pattern to match unsuitable characters for filenames
    invalid_chars = r'[<>:"/\\|?*\']'

    # Replace unsuitable characters with a hyphen
    valid_filename = re.sub(invalid_chars, '-', filename)

    # Truncate the filename if it exceeds the maximum length
    if len(valid_filename) > MAX_FILENAME_LENGTH:
        # Preserve the file extension if present
        if '.' in valid_filename:
            name, extension = valid_filename.rsplit('.', 1)
            truncated_name = name[:MAX_FILENAME_LENGTH - len(extension) - 1]  # Reserve space for the dot
            valid_filename = f"{truncated_name}.{extension}"
        else:
            valid_filename = valid_filename[:MAX_FILENAME_LENGTH]

    return valid_filename


def Aria2Downloader(url, output_file=''):
    if not url.startswith("http"):
        print(f"Invalid URL: {url}")
    _responseFileName = getResponseFileName(url)
    if _responseFileName:
        if not output_file:
            output_file = _responseFileName
        if os.path.exists(output_file):
            print(f"Download File Already Exists: {output_file}")
            return output_file
        else:
            """Download a file using Aria2."""
            print(f"Downloading file: {url}")
            command = f"aria2c -x 16 -s 16 -o \"{output_file}\" {url}"
            status_code = runCommand(command, 1)
            if status_code == 0:
                print(f"\nDownload Successfull: {output_file}\n")
                return output_file
            else:
                print(f"Download Error Status code({status_code}): {url}")
    else:
        print(f"Download Error Response FileName: {url}")
    return url


def mergeMKVs(video_paths, output_dir, max_size_gb):
    if len(video_paths) < 2:
        print(video_paths, f" ==> Can't Merge {len(video_paths)} video")
        return video_paths

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Function to get track IDs using ffmpeg probe
    def get_track_ids(file):
        probe = ffmpeg.probe(file)
        return {stream['index'] for stream in probe['streams']}

    # Get all track IDs across files
    all_track_ids = set.union(*[get_track_ids(file) for file in video_paths])

    # Find files with missing track IDs
    files_to_exclude = []
    valid_files = []

    for file in video_paths:
        file_track_ids = get_track_ids(file)
        if all_track_ids.issubset(file_track_ids):
            valid_files.append(file)
        else:
            files_to_exclude.append(file)

    # If no valid files are left, exit
    if not valid_files:
        print("No valid files to merge. All files have missing track IDs.")
        return files_to_exclude

    # Base output file path
    base_output_path = os.path.join(output_dir, extract_file_info(valid_files[0])[2] + "_merged")

    # Convert max_size_gb to bytes (mkvmerge uses bytes for splitting)
    max_size_bytes = max_size_gb * 1024 * 1024 * 1024

    command = [
        "mkvmerge",
        "-o", f"{base_output_path}.mkv",
        "--split", f"{max_size_bytes}",
    ]

    # Append valid videos sequentially
    for i, video_path in enumerate(valid_files):
        if i == 0:
            command.append(video_path)  # First video
        else:
            command.extend(["+", video_path])  # Append subsequent videos

    # Run the mkvmerge command
    command_str = ' '.join(command)
    runCommand(command_str, 1)

    # Display files that were not merged
    if files_to_exclude:
        print("The following files were not merged due to missing track IDs:")
        for file in files_to_exclude:
            print(f"- {file}")
    else:
        print("All files were merged successfully.")

    return files_to_exclude


def main(filesListStr, output_dir=''):
    Files_List = strToList(filesListStr, ",")
    _File = Files_List[0]

    if len(Files_List) == 0 or (len(Files_List) == 1 and os.path.exists(_File) and _File.endswith('.mkv')):
        print(Files_List, " ==> Give At Least 2 files for Merging")
        exit()

    if len(Files_List) == 1:
        _File = Files_List[0]
        if _File.startswith('http'):
            _File = Aria2Downloader(_File)
        if os.path.exists(_File):
            if patoolib.is_archive(_File):
                _Folder = extractArchive(_File)
                Files_List = getMKVFilesList(_Folder)
            elif os.path.isdir(_File):
                Files_List = getMKVFilesList(_File)

    valid_Files_List = []
    if len(Files_List) > 1:
        for i, valid_File in enumerate(Files_List):
            if valid_File.startswith('http'):
                valid_File = Aria2Downloader(valid_File)
            if os.path.exists(valid_File):
                if valid_File.endswith('.mkv'):
                    valid_File = os.path.abspath(valid_File)
                    valid_Files_List.append(valid_File)
                else:
                    print("File Exists but Unsupported File Format: ", valid_File)
            else:
                print(f"Error FILES {i+1}: {valid_File}")
                exit()
        print("Valid Files are:\n", "\n".join(valid_Files_List))

    if len(valid_Files_List) > 1:
        if JUST_DOWNLOAD:
            exit()
        if not output_dir:
            output_dir = '/content/drive/MyDrive'
        mergeMKVs(valid_Files_List, output_dir)

main(MKV_FILES, OUTPUT_DIR, MAX_FileSize)